In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [ ]:

def SSVQE_all_kstates(hamiltonian, ansatz, params, opt_name='Adam', stepsize=0.1, iterations=100, k=[0], device_name='default.qubit'):
    # Calculate number of qubits required for the given Hamiltonian and create the device.
    qubits = len(hamiltonian.wires)
    device = qml.device(device_name, wires=qubits)
    wires = range(qubits)

    #Define the weights for the cost function.
    weights = np.arange(k+1, 0, -1)

    # State Preparation for orthogonal states.
    def state_prep(state_value):
      for index, value in enumerate(np.binary_repr(state_value, qubits)):
        if value == '1':
          qml.PauliX(index)

    # Cost of each orthogonal state.
    @qml.qnode(device)
    def state_cost(params, state_value):
      state_prep(state_value)
      ansatz(params, wires)
      return qml.expval(hamiltonian)

    # Total cost of the system.
    def total_cost(params):
      cost = 0
      for index in range(k+1):
          cost += weights[index] * state_cost(params, state_value=index)
      return cost

    # Optimizers options.
    optimizers = {
        'Adam': qml.AdamOptimizer(stepsize=stepsize),
        'Adagrad': qml.AdagradOptimizer(stepsize=stepsize),
        'GradientDescent': qml.GradientDescentOptimizer(stepsize=stepsize)
    }

    # Define the Optimizer and initial parameters.
    optimizer = optimizers[opt_name]
    energies = np.zeros((k+1,iterations))

    # Optimization loop.
    for itr in range(iterations):
      params = optimizer.step(total_cost, params)
      for index in range(k+1):
        energy = state_cost(params, state_value=index)
        energies[index][itr] = energy

    # Return List
    Energies = []
    for energy in energies:
        Energies.append([energy[-1], energy])
    return Energies

In [ ]:
def SSVQE_custom_kstates(hamiltonian, ansatz, params, opt_name='Adam', stepsize=0.1, iterations=100, k=[0], device_name='default.qubit'):
    # Calculate number of qubits required for the given Hamiltonian and create the device.
    qubits = len(hamiltonian.wires)
    device = qml.device(device_name, wires=qubits)
    wires = range(qubits)

    #Define the weights for the cost function.
    weights = np.arange(max(k)+1,0,-1)

    # State Preparation for orthogonal states.
    def state_prep(state_value):
      for index, value in enumerate(np.binary_repr(state_value, qubits)):
        if value == '1':
          qml.PauliX(index)

    # Cost of each orthogonal state.
    @qml.qnode(device)
    def state_cost(params, state_value):
      state_prep(state_value)
      ansatz(params, wires)
      return qml.expval(hamiltonian)

    # Total cost of the system.
    def total_cost(params):
      cost = 0
      for index in range(max(k)+1):
          cost += weights[index] * state_cost(params, state_value=index)
      return cost

    # Optimizers options.
    optimizers = {
        'Adam': qml.AdamOptimizer(stepsize=stepsize),
        'Adagrad': qml.AdagradOptimizer(stepsize=stepsize),
        'GradientDescent': qml.GradientDescentOptimizer(stepsize=stepsize)
    }

    # Define the Optimizer and initial parameters.
    optimizer = optimizers[opt_name]
    energies = np.zeros((max(k)+1,iterations))

    # Optimization loop.
    for itr in range(iterations):
      params = optimizer.step(total_cost, params)
      for index in range(max(k)+1):
        if index in k:
          energy = state_cost(params, state_value=index)
          energies[index][itr] = energy

    # Return List
    Energies = []
    for energy in energies:
        Energies.append([energy[-1], energy])
    return Energies

In [ ]:
def block(weights, wires):
    qml.CNOT(wires=[wires[0],wires[1]])
    qml.RY(weights[0], wires=wires[0])
    qml.RY(weights[1], wires=wires[1])

n_wires = 2
n_block_wires = 2
n_params_block = 2
n_blocks = qml.MERA.get_n_blocks(range(n_wires),n_block_wires)
template_weights = [[0.1,-0.3]]*n_blocks

dev= qml.device('default.qubit',wires=range(n_wires))
@qml.qnode(dev)
def circuit(template_weights):
    qml.MERA(range(n_wires),n_block_wires,block, n_params_block, template_weights)
    return qml.expval(qml.PauliZ(wires=1))


#Circuit Anstaz:
def ansatz(params, wires):
    #qml.MERA(range(n_wires),n_block_wires,block, n_params_block, template_weights)
    qml.templates.StronglyEntanglingLayers(params, wires=wires)

In [ ]:
distances = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0]
ssvqe_energies = []
ssvqe_energies_mera = []

for r in distances:
  print('r=',r)
  # Molecule Parameters:
  symbols_lih = ["Li", "H"]

  geometry_lih = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r])

  # Hamiltonian:
  hamiltonian_lih, qubits_lih =  qml.qchem.molecular_hamiltonian(
          symbols_lih,
          geometry_lih,
          active_electrons=2,
          active_orbitals=2
  )

  # Params:
  qubits_lih_tapered = qubits_lih-2
  params_lih = np.random.uniform(0, 2*np.pi, (6, qubits_lih_tapered, 3))
  #params_lih = np.random.uniform(0, 2*np.pi, (6, qubits_lih, 3))


  generators = qml.symmetry_generators(hamiltonian_lih)
  paulixops = qml.paulix_ops(generators, qubits_lih)

  n_electrons = 2
  paulix_sector = qml.qchem.optimal_sector(hamiltonian_lih, generators, n_electrons)

  H_tapered = qml.taper(hamiltonian_lih, generators, paulixops, paulix_sector)
  H_tapered = qml.Hamiltonian(np.real(H_tapered.coeffs), H_tapered.ops)

  energies_all = SSVQE_all_kstates(H_tapered, ansatz, params_lih, stepsize=0.05, k=1)
  #energies_custom = SSVQE_custom_kstates(H_tapered, ansatz, params_lih, stepsize=0.05, k=[1])

  ssvqe_energies.append( energies_all[1][0] )

print(ssvqe_energies)

